### Import dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Extract data

In [2]:
# Define paths
# Data from https://aqs.epa.gov/aqsweb/airdata/download_files.html
path15 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2015.csv'
path16 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2016.csv'
path17 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2017.csv'
path18 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2018.csv'
path19 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2019.csv'
path20 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2020.csv'

In [3]:
# Read in data for six year of AQI data
df15 = pd.DataFrame(pd.read_csv(path15))
df16 = pd.DataFrame(pd.read_csv(path16))
df17 = pd.DataFrame(pd.read_csv(path17))
df18 = pd.DataFrame(pd.read_csv(path18))
df19 = pd.DataFrame(pd.read_csv(path19))
df20 = pd.DataFrame(pd.read_csv(path20))

# Transform data

In [4]:
# Append all six years into one DF
six_years_df = df15.append([df16,df17,df18,df19,df20], ignore_index=True)

In [5]:
# Split "Date" column into "Year", "Month", and "Day" columns for 2020 data
df20['Year'] = [d.split('-')[0] for d in df20['Date']]
df20['Month'] = [d.split('-')[1] for d in df20['Date']]
df20['Day'] = [d.split('-')[2] for d in df20['Date']]

# Create column to hold Month and Day w/o year
# This is to compare specific days across all six years
# TODO: Find a cleaner, more effective way to do this
df20['Month_Day'] = df20['Month'] + '-' + df20['Day']

# Change type from str to int64
df20 = df20.astype({'Year': 'int64','Month': 'int64','Day': 'int64'})

# Drop unneeded columns
df20 = df20.drop(columns={'State Code','Defining Site'})

df20

,State Name,county Name,County Code,Date,AQI,Category,Defining Parameter,Number of Sites Reporting,Year,Month,Day,Month_Day
0,Alabama,Baldwin,3,2020-01-01,48,Good,PM2.5,1,2020,1,1,01-01
1,Alabama,Baldwin,3,2020-01-04,13,Good,PM2.5,1,2020,1,4,01-04
2,Alabama,Baldwin,3,2020-01-07,14,Good,PM2.5,1,2020,1,7,01-07
3,Alabama,Baldwin,3,2020-01-10,39,Good,PM2.5,1,2020,1,10,01-10
4,Alabama,Baldwin,3,2020-01-13,29,Good,PM2.5,1,2020,1,13,01-13
...,...,...,...,...,...,...,...,...,...,...,...,...
41245,Wyoming,Uinta,41,2020-03-27,5,Good,PM10,2,2020,3,27,03-27
41246,Wyoming,Uinta,41,2020-03-28,6,Good,PM10,2,2020,3,28,03-28
41247,Wyoming,Uinta,41,2020-03-29,6,Good,PM10,2,2020,3,29,03-29
41248,Wyoming,Uinta,41,2020-03-30,5,Good,PM10,2,2020,3,30,03-30


In [6]:
# Split "Date" column into "Year", "Month", and "Day" columns for all six years of data
six_years_df['Year'] = [d.split('-')[0] for d in six_years_df['Date']]
six_years_df['Month'] = [d.split('-')[1] for d in six_years_df['Date']]
six_years_df['Day'] = [d.split('-')[2] for d in six_years_df['Date']]

# Create column to hold Month and Day w/o year
# This is to compare specific days across all six years
six_years_df['Month_Day'] = six_years_df['Month'] + '-' + six_years_df['Day']

# Change type from str to int64
six_years_df = six_years_df.astype({'Year': 'int64','Month': 'int64','Day': 'int64'})

# Drop unneeded columns
six_years_df = six_years_df.drop(columns={'State Code','Defining Site'})

In [7]:
# Create DF for just 2020 and 2019
df_19_20 = six_years_df.loc[((six_years_df['Year'] == 2019) | (six_years_df['Year'] == 2020))]
# df_19_20.to_csv('../01_Resources/2019-20_AQI_data.csv')

In [8]:
# Find the five-year avg
five_year_avg = six_years_df.loc[six_years_df['Year'] != 2020].groupby(['County Code','Month_Day'])['AQI'].mean().reset_index()

# Rename column    
five_year_avg = five_year_avg.rename(columns={'AQI':'Five-Year Avg.'})
five_year_avg

,County Code,Month_Day,Five-Year Avg.
0,1,01-01,36.950413
1,1,01-02,38.042735
2,1,01-03,43.327731
3,1,01-04,40.271186
4,1,01-05,38.921739
...,...,...,...
60462,840,12-23,24.000000
60463,840,12-26,8.500000
60464,840,12-27,7.000000
60465,840,12-28,5.000000


In [9]:
# Merge five-year avg. data into 2020 DF
compare_df = pd.merge(df20, five_year_avg,  how='left', 
                      left_on=['County Code','Month_Day'], 
                      right_on = ['County Code','Month_Day'])
compare_df  

,State Name,county Name,County Code,Date,AQI,Category,Defining Parameter,Number of Sites Reporting,Year,Month,Day,Month_Day,Five-Year Avg.
0,Alabama,Baldwin,3,2020-01-01,48,Good,PM2.5,1,2020,1,1,01-01,36.239316
1,Alabama,Baldwin,3,2020-01-04,13,Good,PM2.5,1,2020,1,4,01-04,34.163793
2,Alabama,Baldwin,3,2020-01-07,14,Good,PM2.5,1,2020,1,7,01-07,40.389831
3,Alabama,Baldwin,3,2020-01-10,39,Good,PM2.5,1,2020,1,10,01-10,35.122807
4,Alabama,Baldwin,3,2020-01-13,29,Good,PM2.5,1,2020,1,13,01-13,34.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41245,Wyoming,Uinta,41,2020-03-27,5,Good,PM10,2,2020,3,27,03-27,35.542373
41246,Wyoming,Uinta,41,2020-03-28,6,Good,PM10,2,2020,3,28,03-28,36.393443
41247,Wyoming,Uinta,41,2020-03-29,6,Good,PM10,2,2020,3,29,03-29,40.344828
41248,Wyoming,Uinta,41,2020-03-30,5,Good,PM10,2,2020,3,30,03-30,38.754386


In [ ]:
# compare_df.to_csv('../01_Resources/aqi_2020_5y.csv')

In [11]:
# Chage DF names and drop unsused
df_19_20 = df_19_20.drop(columns={'Year','Month','Day'})
df_19_20 = df_19_20.rename(columns={'State Name':'state',
                                    'county Name':'county',
                                    'County Code':'county_code',
                                    'Date':'date',
                                    'AQI':'aqi',
                                    'Category':'category',
                                    'Defining Parameter':'defining_parameter',
                                    'Number of Sites Reporting':'number_sites',
                                    'Month_Day':'month_day'
                                   })

six_years_df = six_years_df.drop(columns={'Year','Month','Day'})
six_years_df = six_years_df.rename(columns={'State Name':'state',
                                    'county Name':'county',
                                    'County Code':'county_code',
                                    'Date':'date',
                                    'AQI':'aqi',
                                    'Category':'category',
                                    'Defining Parameter':'defining_parameter',
                                    'Number of Sites Reporting':'number_sites',
                                    'Month_Day':'month_day'
                                   })

compare_df = compare_df.drop(columns={'Year','Month','Day'})
compare_df = compare_df.rename(columns={'State Name':'state',
                                    'county Name':'county',
                                    'County Code':'county_code',
                                    'Date':'date',
                                    'AQI':'aqi',
                                    'Category':'category',
                                    'Defining Parameter':'defining_parameter',
                                    'Number of Sites Reporting':'number_sites',
                                    'Month_Day':'month_day',
                                    'Five-Year Avg.':'five_year_avg'
                                   })
compare_df

,state,county,county_code,date,aqi,category,defining_parameter,number_sites,month_day,five_year_avg
0,Alabama,Baldwin,3,2020-01-01,48,Good,PM2.5,1,01-01,36.239316
1,Alabama,Baldwin,3,2020-01-04,13,Good,PM2.5,1,01-04,34.163793
2,Alabama,Baldwin,3,2020-01-07,14,Good,PM2.5,1,01-07,40.389831
3,Alabama,Baldwin,3,2020-01-10,39,Good,PM2.5,1,01-10,35.122807
4,Alabama,Baldwin,3,2020-01-13,29,Good,PM2.5,1,01-13,34.266667
...,...,...,...,...,...,...,...,...,...,...
41245,Wyoming,Uinta,41,2020-03-27,5,Good,PM10,2,03-27,35.542373
41246,Wyoming,Uinta,41,2020-03-28,6,Good,PM10,2,03-28,36.393443
41247,Wyoming,Uinta,41,2020-03-29,6,Good,PM10,2,03-29,40.344828
41248,Wyoming,Uinta,41,2020-03-30,5,Good,PM10,2,03-30,38.754386


# Load data

In [12]:
compare_df['county_state'] = compare_df['county'] + ', ' + compare_df['state']
compare_df

,state,county,county_code,date,aqi,category,defining_parameter,number_sites,month_day,five_year_avg,county_state
0,Alabama,Baldwin,3,2020-01-01,48,Good,PM2.5,1,01-01,36.239316,"Baldwin, Alabama"
1,Alabama,Baldwin,3,2020-01-04,13,Good,PM2.5,1,01-04,34.163793,"Baldwin, Alabama"
2,Alabama,Baldwin,3,2020-01-07,14,Good,PM2.5,1,01-07,40.389831,"Baldwin, Alabama"
3,Alabama,Baldwin,3,2020-01-10,39,Good,PM2.5,1,01-10,35.122807,"Baldwin, Alabama"
4,Alabama,Baldwin,3,2020-01-13,29,Good,PM2.5,1,01-13,34.266667,"Baldwin, Alabama"
...,...,...,...,...,...,...,...,...,...,...,...
41245,Wyoming,Uinta,41,2020-03-27,5,Good,PM10,2,03-27,35.542373,"Uinta, Wyoming"
41246,Wyoming,Uinta,41,2020-03-28,6,Good,PM10,2,03-28,36.393443,"Uinta, Wyoming"
41247,Wyoming,Uinta,41,2020-03-29,6,Good,PM10,2,03-29,40.344828,"Uinta, Wyoming"
41248,Wyoming,Uinta,41,2020-03-30,5,Good,PM10,2,03-30,38.754386,"Uinta, Wyoming"


In [13]:
# Import dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, select, join

In [ ]:
# Base = declarative_base()

# Create table schema
# class Aqi(Base):
#     # Tell SQLAlchemy what the table name is and if there are any table-specific arguments
#     __tablename__ = 'daily_aqi'
#     __table_args__ = {'sqlite_autoincrement': True}
#     # Tell SQLAlchemy the name of column and attributes
#     id = Column(Integer, primary_key=True, nullable=False) 
#     state = Column(VARCHAR(40))
#     county = Column(VARCHAR)
#     county_code = Column(Integer)
#     date = Column(Date)
#     aqi = Column(Integer)
#     category = Column(VARCHAR)
#     defining_parameter = Column(VARCHAR)
#     number_sites = Column(Integer)
#     year = Column(Integer)
#     month = Column(Integer)
#     day = Column(Integer)
#     month_day = Column(VARCHAR)
    
# # Create engine
# engine = create_engine('sqlite:///aqi.db')
# Base.metadata.create_all(engine)

# # Create SQLite DB from the six-year/Jan-June DF
# before_june_df.to_sql(con=engine, index_label='id', name=Aqi.__tablename__, if_exists='replace')

In [15]:
# Create engine
# Note: User will need to supply their own PostgreSQL password under variable below
user = 'postgres'
host = 'localhost'
password = 'postgres-2002'
port = '5432'
db = 'avg_aqi'
uri = f'postgresql://{user}:{password}@{host}:{port}/{db}'
uri
engine = create_engine(uri)

# Export content to SQL table
df_19_20.to_sql(name='aqi_2019_2020', con=engine, if_exists='replace', index=False)
six_years_df.to_sql(name='daily_aqi', con=engine, if_exists='replace', index=False)
compare_df.to_sql(name='aqi_2020', con=engine, if_exists='replace', index=False)

# Exploration/Visualization

In [ ]:
# the following are not availble at the moment

In [ ]:
# Create DFs for the avg AQI for 2019 and 2020 by date
mean_aqi_19 = six_years_df.loc[six_years_df['Year']==2019].groupby('Date')['AQI'].mean()
mean_aqi_20 = six_years_df.loc[six_years_df['Year']==2020].groupby('Date')['AQI'].mean()

In [ ]:
mean_aqi_19.plot(kind='line',figsize=(17,12))
plt.plot(mean_aqi_20)
plt.ylabel('Avg. Air Quality Index')
plt.axvline(x='2020-02-25',c='red',alpha=.75)
plt.axvline(x='2020-05-01',c='g',alpha=.75)
plt.annotate('San Fransico declares state of emergency', xy=('2020-02-26', 65), c='r')
plt.annotate('Texas starts "reopening"', xy=('2020-04-08', 13), c='g')
plt.legend(["2019","2020"])
plt.title("2019 vs. 2020 Avg. Daily Air Quality Index")

In [ ]:
# Create DF to hold avg AQI for 2020 by day of the year
avg_2020 = six_years_df.loc[six_years_df['Year']==2020].groupby('Month_Day')['AQI'].mean().reset_index()

# Create DF to hold avg AQI for 2015-2019 by day of the year
avg_historic = six_years_df.loc[six_years_df['Month']<6]
avg_historic = avg_historic.loc[six_years_df.Year!=2020].groupby('Month_Day')['AQI'].mean().reset_index()

avg_2020

In [ ]:
# TODO: fix this viz
# Need to figure out how to correctly compare dates across years
avg_2020.plot(kind='line',figsize=(15,10))
plt.plot(avg_historic['Month_Day'], avg_historic['AQI'])
plt.legend(["2020", "Last 5 Years"])